In [42]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import os
import numpy as np
import cv2
import random



tf.logging.set_verbosity(tf.logging.INFO)

# emotion coding: 0=neutral, 1=anger, 2=contempt, 3=disgust, 4=fear, 5=happy, 6=sadness, 7=surprise
# e_labels = [0, 1, 2, 3, 4, 5, 6, 7]
# this is not in the domain of our loss function, so we will increase all values by 1

In [43]:
emotions = []
for path, name, files in os.walk('Emotion'):
    for file in files:
        if(file):
            with open(os.path.join(path, file)) as curf:
                emotions.append((os.path.join(path, file), file[:-4], int(curf.read().strip(' ')[:1]) + 1))

print(emotions[0])
img = os.path.join('aligned', emotions[0][1][:-8] + '.png')
print(cv2.imread(img, 0))

print(len(emotions))

('Emotion\\S005\\001\\S005_001_00000011_emotion.txt', 'S005_001_00000011_emotion', 4)
[[  0   0   0 ...,   0   0   0]
 [  0   0   0 ...,   0   0   0]
 [  0   0   0 ...,   0   0   0]
 ..., 
 [187 189 189 ...,  78  84  92]
 [186 188 186 ...,  87  93  97]
 [192 189 185 ...,  92  98 100]]
327


In [51]:
# let's pick 40 eval samples, too bad this dataset is so small
evals = []
for k in range(40):
    evals.append(random.randrange(326))
print(len(evals))

40


In [45]:
# want shape (num images, 115,600)
data = []
labels = []
eval_data = []
eval_labels = []


for i in range(len(emotions)):
    im = os.path.join(os.path.join('aligned', emotions[i][1][:-8] + '.png'))
    im_lst = cv2.imread(im, 0)
    if(i in evals):
        eval_data.append(im_lst.flatten())
        eval_labels.append(emotions[i][2])
    else:
        data.append(im_lst.flatten())
        labels.append(emotions[i][2])

npdata = np.array(data, dtype=np.float32)
nplabels = np.array(labels, dtype=np.int32)
npeval_data = np.array(eval_data, dtype=np.float32)
npeval_labels = np.array(eval_labels, dtype=np.int32)

assert not np.any(np.isnan(npdata))
assert not np.any(np.isnan(nplabels))

print(npdata.shape)
npdata

(288, 115600)


array([[   0.,    0.,    0., ...,   92.,   98.,  100.],
       [  14.,   13.,   12., ...,    0.,    0.,    0.],
       [  12.,   11.,   11., ...,  145.,  146.,  146.],
       ..., 
       [  63.,   62.,   61., ...,   56.,   55.,   55.],
       [  57.,   59.,   64., ...,   35.,   39.,   45.],
       [  51.,   54.,   56., ...,   39.,   38.,   39.]], dtype=float32)

In [46]:
onehot_labels = tf.one_hot(indices=tf.cast(nplabels[:10], tf.int32), depth=8)
print(onehot_labels.shape)

(10, 8)


In [47]:
# we have 340x340px images
def cnn_model(features, labels, mode):
    input_layer = tf.reshape(features['x'], [-1, 340, 340, 1])
    
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=64,
        kernel_size=[3, 3],
        padding="same",
        activation=tf.nn.relu)
    # shape [batch size, 340, 340, 64]
    print("conv1", conv1.shape)
    
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
    # shape [batch size, 170, 170, 64]
    print("pool1", pool1.shape)
    
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[3, 3],
        padding="same",
        activation=tf.nn.relu)
    # shape [batch size, 170, 170, 128]
    print("conv2", conv2.shape)
    
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2, padding="same")
    # shape [batch size, 81, 81, 64]
    
    print("pool2", pool2.shape)
    
    # padded_pool2 = tf.pad(pool2, [[1, 1], [1, 1]], "CONSTANT")
    # print(tf.shape(padded_pool2))
    s = pool2.shape
    pool2_flat = tf.reshape(pool2, [-1, int(s[1]) * int(s[2]) * int(s[3])])
    print("pool2_flat", pool2_flat.shape)
    
    dense=tf.layers.dense(inputs=pool2_flat, units=512, activation=tf.nn.relu)
    print("dense", dense.shape)
    
    dropout = tf.layers.dropout(inputs=dense, rate=.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    print("dropout", dropout.shape)
    
    logits = tf.layers.dense(inputs=dropout, units=8)
    print("logits", logits.shape)

    predictions = {
        "classes":tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=8)
    loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=.001)
        train_op=optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    eval_metric_ops = {
        "accuracy":tf.metrics.accuracy(
            labels=labels, predictions=predictions["classes"])
    }
    
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)



In [48]:
ck_classifier = tf.estimator.Estimator(model_fn=cnn_model, model_dir="models")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'models', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


In [49]:
tensors_to_log = {"probabilities":"softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x":npdata},
    y=nplabels,
    batch_size=1,
    num_epochs=None,
    shuffle=True)

ck_classifier.train(
                input_fn=train_input_fn,
                steps=5000,
                hooks=[logging_hook])

conv1 (1, 340, 340, 64)
pool1 (1, 170, 170, 64)
conv2 (1, 170, 170, 64)
pool2 (1, 85, 85, 64)
pool2_flat (1, 462400)
dense (1, 512)
dropout (1, 512)
logits (1, 8)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from models\model.ckpt-7002
INFO:tensorflow:Saving checkpoints for 7003 into models\model.ckpt.
INFO:tensorflow:probabilities = [[ 0.07411177  0.07418785  0.15190901  0.09814024  0.17269799  0.11221687
   0.19982083  0.11691548]]
INFO:tensorflow:loss = 1.75621, step = 7003
INFO:tensorflow:probabilities = [[ 0.07389759  0.07397316  0.15148813  0.09803266  0.17393468  0.11201548
   0.19998036  0.11667798]] (2.537 sec)
INFO:tensorflow:global_step/sec: 19.9685
INFO:tensorflow:probabilities = [[ 0.07368401  0.07375906  0.15137053  0.09811986  0.17429803  0.11203692
   0.19994144  0.11679017]] (2.472 sec)
INFO:tensorflow:loss = 1.60973, step = 7103 (4.968 sec)
INFO:tensorflow:probabilities = [[ 0.07344458  0.07351912  0.15134713  0.09787755  0.1747686 

INFO:tensorflow:loss = 1.53957, step = 9003 (4.946 sec)
INFO:tensorflow:probabilities = [[ 0.06711742  0.06717703  0.15302329  0.09353173  0.17854826  0.11030433
   0.21539189  0.11490592]] (2.465 sec)
INFO:tensorflow:global_step/sec: 20.184
INFO:tensorflow:probabilities = [[ 0.06699492  0.06705423  0.1531578   0.09351812  0.17918837  0.11019578
   0.21512479  0.11476601]] (2.489 sec)
INFO:tensorflow:loss = 1.71932, step = 9103 (4.954 sec)
INFO:tensorflow:probabilities = [[ 0.06681052  0.06686946  0.1526906   0.09323049  0.1809164   0.10987527
   0.21508755  0.11451969]] (2.500 sec)
INFO:tensorflow:global_step/sec: 19.8591
INFO:tensorflow:probabilities = [[ 0.06668593  0.06674457  0.15312164  0.09339861  0.18063504  0.10965351
   0.21481217  0.11494856]] (2.536 sec)
INFO:tensorflow:loss = 1.71128, step = 9203 (5.036 sec)
INFO:tensorflow:probabilities = [[ 0.0665825   0.06664085  0.15329488  0.09340905  0.18094695  0.10946716
   0.2148208   0.11483777]] (2.488 sec)
INFO:tensorflow:globa

INFO:tensorflow:probabilities = [[ 0.06146229  0.06150996  0.155825    0.08998825  0.18722302  0.1074213
   0.22313684  0.1134334 ]] (2.412 sec)
INFO:tensorflow:global_step/sec: 20.7028
INFO:tensorflow:probabilities = [[ 0.06129678  0.06134415  0.15576078  0.08970782  0.18722466  0.10752878
   0.22385105  0.11328594]] (2.420 sec)
INFO:tensorflow:loss = 1.67545, step = 11203 (4.831 sec)
INFO:tensorflow:probabilities = [[ 0.06113636  0.06118345  0.15539679  0.08952501  0.18761311  0.10753598
   0.22457403  0.11303525]] (2.409 sec)
INFO:tensorflow:global_step/sec: 20.715
INFO:tensorflow:probabilities = [[ 0.06104627  0.06109313  0.15598775  0.08962355  0.18746458  0.10755789
   0.22442473  0.11280204]] (2.418 sec)
INFO:tensorflow:loss = 2.22973, step = 11303 (4.826 sec)
INFO:tensorflow:probabilities = [[ 0.06096737  0.06101401  0.15551506  0.09000695  0.18697701  0.10759836
   0.22499365  0.11292756]] (2.414 sec)
INFO:tensorflow:global_step/sec: 20.7279
INFO:tensorflow:probabilities = [[ 

In [50]:
eval_input_fn=tf.estimator.inputs.numpy_input_fn(
    x={"x":npeval_data},
    y=npeval_labels,
    batch_size=1,
    num_epochs=1,
    shuffle=False)

eval_results=ck_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

conv1 (?, 340, 340, 64)
pool1 (?, 170, 170, 64)
conv2 (?, 170, 170, 64)
pool2 (?, 85, 85, 64)
pool2_flat (?, 462400)
dense (?, 512)
dropout (?, 512)
logits (?, 8)
INFO:tensorflow:Starting evaluation at 2017-09-21-20:01:09
INFO:tensorflow:Restoring parameters from models\model.ckpt-12002
INFO:tensorflow:Finished evaluation at 2017-09-21-20:01:11
INFO:tensorflow:Saving dict for global step 12002: accuracy = 0.179487, global_step = 12002, loss = 1.42763
{'accuracy': 0.17948718, 'loss': 1.4276333, 'global_step': 12002}
